## 1. Importación de librerías y cargado de datos

In [ ]:
import pandas as pd
import numpy as np

print(F"Importación ok. Versión de Pandas instalada = {pd.__version__}")
print(F"Importación ok. Versión de NumPy instalada = {np.__version__}")

In [ ]:
df = pd.read_csv("datos_ejemplo.csv")
df

In [ ]:
print("📊 DATOS ORIGINALES")
print(df.head())
print(f"\nShape: {df.shape}")

## 2. Indexación Jerárquica

In [ ]:
# Crear índice jerárquico con Región y Producto
df_jerarquico = df.set_index(['Región', 'Producto']).sort_index()  # ✅ Ordenar el índice
print("DataFrame con índice jerárquico (Región, Producto):")
df_jerarquico.head(8)

In [ ]:
# Acceso a datos específicos
print("\n📍 Acceso a datos del Centro:")
df_jerarquico.loc['Centro']

In [ ]:
print("\n📍 Acceso específico - Centro, Producto A:")
df_jerarquico.loc[('Centro', 'A')]

## 3. Agrupamiento de datos con groupby()

In [ ]:
# Agrupamiento básico por región
print("💰 Ventas totales por región:")
ventas_region = df.groupby('Región')['Ventas'].sum()
print(ventas_region)

In [ ]:
print("\n📦 Estadísticas de unidades por producto:")
stats_producto = df.groupby('Producto')['Unidades'].agg(['mean', 'sum', 'count', 'min', 'max'])
print(stats_producto)

In [ ]:
print("\n📅 Ventas promedio por mes:")
    ventas_mes = df.groupby('Mes')['Ventas'].mean().round(2)
print(ventas_mes)

In [ ]:
# Aplicar múltiples funciones a la vez
print("📊 Análisis completo por región:")
analisis_region = df.groupby('Región').agg({
    'Ventas': ['sum', 'mean', 'count'],
    'Unidades': ['sum', 'max', 'min']
}).round(2)
print(analisis_region)

In [ ]:
print("\n📈 Resumen por producto:")
resumen_producto = df.groupby('Producto').agg({
    'Ventas': ['sum', 'mean'],
    'Unidades': 'sum'
}).round(2)
print(resumen_producto)

## 4. Pivoteo de datos

In [ ]:
# Crear tabla pivote: Regiones vs Productos (suma de ventas)
print("🔄 Tabla pivote - Ventas por Región y Producto:")
pivot_ventas = df.groupby(['Región', 'Producto'])['Ventas'].sum().unstack(fill_value=0)
print(pivot_ventas)

In [ ]:
print("\n🔄 Tabla pivote - Unidades por Mes y Región:")
pivot_unidades = df.groupby(['Mes', 'Región'])['Unidades'].sum().unstack(fill_value=0)
print(pivot_unidades)

## 5. Despivoteo de datos

In [ ]:
# Despivotear la tabla anterior
print("🔄 Despivoteo de la tabla de ventas:")
df_melted = pivot_ventas.reset_index().melt(
    id_vars=['Región'],
    var_name='Producto',
    value_name='Ventas_Total'
)
print(df_melted[df_melted['Ventas_Total'] > 0])  # Solo mostrar valores > 0

In [ ]:
print("\n🔄 Despivoteo completo del DataFrame original:")
df_melt_completo = df.melt(
    id_vars=['Región', 'Producto', 'Mes'],
    value_vars=['Ventas', 'Unidades'],
    var_name='Métrica',
    value_name='Valor'
)
print(df_melt_completo.head(8))

## 6. Combinación de datos

In [ ]:
# Crear DataFrames adicionales para demostrar combinaciones
df1 = df[df['Región'] == 'Centro'][['Región', 'Producto', 'Ventas']].head(3)
df2 = df[df['Región'] == 'Norte'][['Región', 'Producto', 'Ventas']].head(3)

print("DataFrame 1 (Centro):")
print(df1)
print("\nDataFrame 2 (Norte):")
print(df2)

In [ ]:
# CONCATENACIÓN
print("\nCONCATENACIÓN con concat():")
df_concat = pd.concat([df1, df2], ignore_index=True)
print(df_concat)

In [ ]:
# Crear DataFrame de información adicional para merge
info_productos = pd.DataFrame({
    'Producto': ['A', 'B', 'C'],
    'Categoría': ['Electrónicos', 'Ropa', 'Hogar'],
    'Precio_Unitario': [25.50, 80.00, 45.75]
})

print("DataFrame de información de productos:")
print(info_productos)

In [ ]:
# MERGE - diferentes tipos de uniones
print("\nMERGE - Unión INNER:")
df_inner = df.merge(info_productos, on='Producto', how='inner')
print(df_inner[['Región', 'Producto', 'Ventas', 'Categoría', 'Precio_Unitario']].head(5))

In [ ]:
print("\nMERGE - Unión LEFT:")
df_left = df.merge(info_productos, on='Producto', how='left')
print(f"Filas originales: {len(df)}, Filas después del merge: {len(df_left)}")

In [ ]:
# Ejemplo práctico: Calcular ingresos totales
print("\nEJEMPLO PRÁCTICO - Cálculo de ingresos:")
df_ingresos = df.merge(info_productos, on='Producto', how='left')
df_ingresos['Ingresos_Calculados'] = df_ingresos['Unidades'] * df_ingresos['Precio_Unitario']
df_ingresos[['Región', 'Producto', 'Mes', 'Ventas', 'Unidades', 'Precio_Unitario', 'Ingresos_Calculados']].head()